# OptiGuide Example




Here we give a simple example, as designed and illustrated in the [OptiGuide paper](https://arxiv.org/abs/2307.03875).
While the original paper is designed specifically for supply chain optimization, the general framework can be easily adapted to other applications with coding capacity.




## OptiGuide for Supply Chain Optimization: System Design Overview

The original system design for OptiGuide, tailored for supply chain optimization, is presented below.

The collaboration among three agents -- Coder, Safeguard, and Interpreter -- lies at the core of this system. They leverage a set of external tools and a large language model (LLM) to address users' questions related to supply chain applications. For a comprehensive understanding of the design and data flow, detailed information can be found in the original [paper](https://arxiv.org/abs/2307.03875).


![optiguide system](https://www.beibinli.com/docs/optiguide/optiguide_system.png)


## New Implementation



![](new_design.png)

Advantages of this multi-agent design with autogen:
- Collaborative Problem Solving: The collaboration among the user proxy agent and the assistant agents fosters a cooperative problem-solving environment. The agents can share information and knowledge, allowing them to complement each other's abilities and collectively arrive at better solutions. On the other hand, the Safeguard acts as a virtual adversarial checker, which can perform another safety check pass on the generated code.

- Modularity: The division of tasks into separate agents promotes modularity in the system. Each agent can be developed, tested, and maintained independently, simplifying the overall development process and facilitating code management.

- Memory Management: The OptiGuide agent's role in maintaining memory related to user interactions is crucial. The memory retention allows the agents to have context about a user's prior questions, making the decision-making process more informed and context-aware.



In [74]:
# Install Required Packages
%pip install optiguide
%pip install flaml[openai]
%pip install pyautogen

Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for openai==0.27.8 from https://files.pythonhosted.org/packages/67/78/7588a047e458cb8075a4089d721d7af5e143ff85a2388d4a28c530be0494/openai-0.27.8-py3-none-any.whl.metadata
  Using cached openai-0.27.8-py3-none-any.whl.metadata (13 kB)
Using cached openai-0.27.8-py3-none-any.whl (73 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.3.5
    Uninstalling openai-1.3.5:
      Successfully uninstalled openai-1.3.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyautogen 0.2.0 requires openai~=1.2, but you have openai 0.27.8 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for openai~=1.2 from https://files.pythonhosted.org/packages/30/1d/27c3571504fb6fb1e9f7c906d93590

In [3]:
# test Gurobi installation
import gurobipy as gp
from gurobipy import GRB
from eventlet.timeout import Timeout

# import auxillary packages
import requests  # for loading the example source code
import openai

# import flaml and autogen
from flaml import autogen
from flaml.autogen.agentchat import Agent, UserProxyAgent
from optiguide.optiguide import OptiGuideAgent

from flaml.autogen.agentchat import AssistantAgent
from flaml.autogen.agentchat.agent import Agent
from flaml.autogen.code_utils import extract_code

# %pip install flaml[openai]

In [4]:
autogen.oai.ChatCompletion.start_logging()

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST.json",
    file_location=".",
    filter_dict={
        "model": {
            #"gpt-4",
            #"gpt4",
            #"gpt-4-32k",
            #"gpt-4-32k-0314",
            #"gpt-3.5-turbo"
            "gpt-3.5-turbo-16k"
            #"gpt-3.5-turbo-0301",
            #"chatgpt-35-turbo-0301",
            #"gpt-35-turbo-v0301",
        }
    }
)
config_list

[]

Now, let's import the source code (loading from URL) and also some training examples (defined as string blow).

In [5]:
# Get the source code of our coffee example
code_url = "https://raw.githubusercontent.com/microsoft/OptiGuide/main/benchmark/application/coffee.py"
response  = requests.get(code_url)


# Check if the request was successful
if response.status_code == 200:
    # Get the text content from the response
    code = response.text
else:
    raise RuntimeError("Failed to retrieve the file.")


#code = open( 'Battery_Optimization_Shen_Lapp_Poser.ipynb', "r").read() # for local files


# show the first head and tail of the source code
print("\n".join(code.split("\n")[:10]))
print(".\n" * 3)
print("\n".join(code.split("\n")[-10:]))

code

import time
from gurobipy import GRB, Model

# Example data

capacity_in_supplier = {'supplier1': 150, 'supplier2': 50, 'supplier3': 100}

shipping_cost_from_supplier_to_roastery = {
    ('supplier1', 'roastery1'): 5,
    ('supplier1', 'roastery2'): 4,
.
.
.

# Solve
m.update()
model.optimize()

print(time.ctime())
if m.status == GRB.OPTIMAL:
    print(f'Optimal cost: {m.objVal}')
else:
    print("Not solved to optimality. Optimization status:", m.status)



'import time\nfrom gurobipy import GRB, Model\n\n# Example data\n\ncapacity_in_supplier = {\'supplier1\': 150, \'supplier2\': 50, \'supplier3\': 100}\n\nshipping_cost_from_supplier_to_roastery = {\n    (\'supplier1\', \'roastery1\'): 5,\n    (\'supplier1\', \'roastery2\'): 4,\n    (\'supplier2\', \'roastery1\'): 6,\n    (\'supplier2\', \'roastery2\'): 3,\n    (\'supplier3\', \'roastery1\'): 2,\n    (\'supplier3\', \'roastery2\'): 7\n}\n\nroasting_cost_light = {\'roastery1\': 3, \'roastery2\': 5}\n\nroasting_cost_dark = {\'roastery1\': 5, \'roastery2\': 6}\n\nshipping_cost_from_roastery_to_cafe = {\n    (\'roastery1\', \'cafe1\'): 5,\n    (\'roastery1\', \'cafe2\'): 3,\n    (\'roastery1\', \'cafe3\'): 6,\n    (\'roastery2\', \'cafe1\'): 4,\n    (\'roastery2\', \'cafe2\'): 5,\n    (\'roastery2\', \'cafe3\'): 2\n}\n\nlight_coffee_needed_for_cafe = {\'cafe1\': 20, \'cafe2\': 30, \'cafe3\': 40}\n\ndark_coffee_needed_for_cafe = {\'cafe1\': 20, \'cafe2\': 20, \'cafe3\': 100}\n\ncafes = list(s

In [6]:
# Extract code from jupyter notebook

import nbformat

def extract_code_from_notebook(notebook_path):
    try:
        with open(notebook_path, 'r', encoding='utf-8') as notebook_file:
            notebook_content = nbformat.read(notebook_file, as_version=4)

        code_cells = []
        for cell in notebook_content['cells']:
            if cell['cell_type'] == 'code':
                code_cells.append(cell['source'])

        code_as_string = '\n'.join(code_cells)
        return code_as_string
    except FileNotFoundError:
        print(f"Error: Notebook file '{notebook_path}' not found.")
        return None
    except Exception as e:
        print(f"Error reading notebook: {e}")
        return None

# Example usage
notebook_path = 'Battery_Optimization_Shen_Lapp_Poser.ipynb'
our_code = extract_code_from_notebook(notebook_path)

our_code


'!pip install webdriver_manager\nimport os\nimport re\nimport time\nimport numpy as np\nimport pandas as pd\nimport gurobipy as gp\nimport sys\nfrom gurobipy import GRB\nfrom datetime import datetime\nfrom selenium import webdriver\nfrom webdriver_manager.chrome import ChromeDriverManager\nparameters = {\n    \'name\': \'ElectricityArbitrage\',\n    \'generator_name\': \'ADK HUDSON___FALLS\',\n    \'start_date\': None,\n    \'num_periods\': 24,\n    \'num_markets\': 1,\n    \'num_batteries\': 8,\n    \'battery_capacity\': 100,\n    \'charge_loss\': 0.95,\n    \'max_charge\': 50,\n    \'max_discharge\': 100\n}\n\n# OPTIGUIDE DATA CODE GOES HERE\ndef extract_date(file_path):\n    match = re.match(r\'^(\\d+)\', file_path.split(\'/\')[4])\n\n    return match.group(1)\ndef get_prices(start_date, generator_name):\n    if start_date == None:\n        start_date = datetime.today()\n        start_date = start_date.strftime("%Y%m%d")\n\n    dates = [start_date]\n\n    driver_path = ChromeDriverM

In [7]:
# In-context learning examples.
example_qa = """
----------
Question: What if the battery charge capacity was only 70 percent instead of 95 percent?
Answer Code:
```python
parameters['charge_loss'] = .7
```
----------
Question: What if we had 20 batteries instead of 8?
Answer Code:
```python
 parameters['num_batteries'] = 8
```

"""

In [8]:
our_code.find( '# OPTIGUIDE DATA GOES HERE')

-1

Now, let's create an OptiGuide agent and also a user.

For the OptiGuide agent, we only allow "debug_times" to be 1, which means it can debug its answer once if it encountered errors.

In [9]:
%%capture
agent = OptiGuideAgent(
    name="OptiGuide Super sexy team Example",
    source_code=our_code,
    debug_times=1,
    example_qa="",
    llm_config={
        "request_timeout": 600,
        "seed": 42,
        "config_list": config_list,
    }
)

user = UserProxyAgent(
    "user", max_consecutive_auto_reply=0,
    human_input_mode="NEVER", code_execution_config=False
)

# Get open ai key from txt file
api_path = '../credentials/openai_api_key.txt'

def read_api_key(file_path):
    try:
        with open(file_path, 'r') as file:
            api_key = file.read().strip()
        return api_key
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found.")
        return None
    
openai.api_key = read_api_key(api_path)
openai.api_key


Now, let's create a user's question.

In [12]:
user.initiate_chat(agent, message='What would the profit be if we increased the max charge to 70?')

user (to OptiGuide Super sexy team Example):

What would the profit be if we increased the max charge to 70?

--------------------------------------------------------------------------------
OptiGuide Super sexy team Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide Super sexy team Example):

```python
parameters['max_charge'] = 70
[model, buy, sell] = run(parameters)
print_model(model, buy, sell, parameters)
```

--------------------------------------------------------------------------------
OptiGuide Super sexy team Example (to safeguard):


--- Code ---
parameters['max_charge'] = 70
[model, buy, sell] = run(parameters)
print_model(model, buy, sell, parameters)

--- One-Word Answer: SAFE or DANGER ---


--------------------------------------------------------------------------------
safeguard (to OptiGuide Super sexy team Example):

SAFE

--------------------------------------------------------

In [10]:
user.initiate_chat(agent, message="What is the impact of supplier1 being able to supply only half the quantity at present?")

user (to OptiGuide Coffee Example):

What is the impact of supplier1 being able to supply only half the quantity at present?

--------------------------------------------------------------------------------
OptiGuide Coffee Example (to writer):


Answer Code:


--------------------------------------------------------------------------------
writer (to OptiGuide Coffee Example):

```python
# Adjust the supplier capacity
capacity_in_supplier['supplier1'] /= 2

# Update supply constraints
for s in set(i[0] for i in shipping_cost_from_supplier_to_roastery.keys()):
    if s == 'supplier1':
        model.remove(model.getConstrByName(f"supply_{s}"))
        model.addConstr(sum(x[i] for i in shipping_cost_from_supplier_to_roastery.keys() if i[0] == s) <= capacity_in_supplier[s], f"supply_{s}")
# Rerun the optimization
model.optimize()

if model.status == GRB.OPTIMAL:
    print(f'Optimal cost after supplier1 supply reduction: {model.objVal}')
else:
    print("Not solved to optimality. Optimizat